In [1]:
import re
import json
import requests
from datetime import datetime, timedelta
import torch

In [2]:
api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'

In [3]:
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

def get_forecast(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

In [4]:
def extract_intent_data(intent_response):
    json_pattern = re.compile(r'\{.*\}')
    json_match = json_pattern.search(intent_response)
    if json_match:
        intent_data_str = json_match.group()
        intent_data_str = intent_data_str.replace("'", "\"")
        try:
            return json.loads(intent_data_str)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    return {}

def fetch_weather_data(intent, city):
    if intent == "current_weather":
        return get_weather(city)
    elif intent == "forecast" or intent == 'forecast_weather':
        return get_forecast(city)
    return None

def parse_forecast(api_response, city):
    if not api_response:
        return "Sorry, I couldn't fetch the weather information."

    forecast_list = api_response.get('list', [])
    if not forecast_list:
        return f"Sorry, I couldn't fetch the weather information for {city}."

    tomorrow_date = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
    filtered_forecasts = [forecast for forecast in forecast_list if tomorrow_date in forecast['dt_txt']]
    
    if filtered_forecasts:
        selected_forecast = filtered_forecasts[0]
        forecast_temp = selected_forecast['main']['temp']
        forecast_weather = selected_forecast['weather'][0]['description']
        wind_speed = selected_forecast['wind']['speed']
        humidity = selected_forecast['main']['humidity']

        return (
            f"The forecast for {city} on {tomorrow_date} is {forecast_weather} with a temperature of {forecast_temp}°C, "
            f"wind speed of {wind_speed} meters per second, and humidity of {humidity}%."
        )
    return f"Sorry, I couldn't fetch the weather information for {city} for tomorrow."

def parse_current_weather(api_response, city):
    if not api_response:
        return "Sorry, I couldn't fetch the weather information."

    main = api_response.get('main', {})
    weather = api_response.get('weather', [{}])[0]
    wind = api_response.get('wind', {})
    
    temp = main.get('temp')
    description = weather.get('description')
    wind_speed = wind.get('speed')
    humidity = main.get('humidity')

    return (
        f"The current weather in {city} is {description} with a temperature of {temp}°C, "
        f"wind speed of {wind_speed} meters per second, and humidity of {humidity}%."
    )

# def generate_assistant_response(user_input, api_response, model, tokenizer):
#     messages = [
#         {"from": "human", "value": user_input},
#         {"from": "api", "value": api_response}
#     ]
#     inputs = tokenizer.apply_chat_template(
#         messages,
#         tokenize=True,
#         add_generation_prompt=True,
#         return_tensors="pt",
#     ).to("cuda")

#     outputs = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
#     return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [5]:
def generate_assistant_response(user_input, api_response, model, tokenizer):
    messages = [
        {"from": "human", "value": user_input},
        {"from": "api", "value": api_response}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Extract only the relevant part
    return extract_relevant_part(response_text)

def extract_relevant_part(response_text):
    pattern = re.compile(r'The (current )?weather in .+ is .+ with a temperature of .+°C, wind speed of .+ meters per second, and humidity of .+%.')
    match = pattern.search(response_text)
    if match:
        return match.group()
    return response_text


In [6]:
# def inference_pipeline(user_input, model, tokenizer):
#     # Step 1: Intent Identification
#     messages = [{"from": "human", "value": user_input}]
#     inputs = tokenizer.apply_chat_template(
#         messages,
#         tokenize=True,
#         add_generation_prompt=True,
#         return_tensors="pt",
#     ).to("cuda")

#     intent_output = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
#     intent_response = tokenizer.batch_decode(intent_output, skip_special_tokens=True)[0]

#     # Extract intent data
#     intent_data = extract_intent_data(intent_response)
#     intent = intent_data.get("intent")
#     city = intent_data.get("entities", {}).get("city")
#     date = intent_data.get("entities", {}).get("date")

#     # Step 2: Fetch weather data from API
#     api_response = fetch_weather_data(intent, city)
#     if intent == "current_weather":
#         parsed_response = parse_current_weather(api_response, city)
#     else:
#         parsed_response = parse_forecast(api_response, city)

#     # Step 3: Generate the final assistant response using LLM
#     final_response = generate_assistant_response(user_input, parsed_response, model, tokenizer)
#     return final_response

In [7]:
def inference_pipeline(user_input, model, tokenizer):
    # Step 1: Intent Identification
    messages = [{"from": "human", "value": user_input}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    intent_output = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
    intent_response = tokenizer.batch_decode(intent_output, skip_special_tokens=True)[0]

    # Extract intent data
    intent_data = extract_intent_data(intent_response)
    intent = intent_data.get("intent")
    city = intent_data.get("entities", {}).get("city")
    date = intent_data.get("entities", {}).get("date")

    # Step 2: Fetch weather data from API
    api_response = fetch_weather_data(intent, city)
    if intent == "current_weather":
        parsed_response = parse_current_weather(api_response, city)
    else:
        parsed_response = parse_forecast(api_response, city)

    # Step 3: Generate the final assistant response using LLM
    final_response = generate_assistant_response(user_input, parsed_response, model, tokenizer)
    return final_response

In [10]:
from unsloth import FastLanguageModel

# model ,tokenizer = FastLanguageModel.from_pretrained("../models/trained")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "../models/trained",
    max_seq_length = 512,
    dtype = None,
    load_in_4bit = True,
    device_map="auto"
)


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.811 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
user_input = "What is the weather in Kalyani on 26 June 2024?"
final_response = inference_pipeline(user_input, model, tokenizer)
print("Final Assistant Response:", final_response)

In [ ]:
user_input = "What is the weather in Kalyani on Today?"
final_response = inference_pipeline(user_input, model, tokenizer)
print("Final Assistant Response:", final_response)

In [ ]:
user_input = "What is the weather in Kalyani evening?"
final_response = inference_pipeline(user_input, model, tokenizer)
print("Final Assistant Response:", final_response)

In [ ]:
# final_response

In [ ]:
user_input = "What is the weather in Kalyani Tommorow Afternoon"
final_response = inference_pipeline(user_input, model, tokenizer)
print("Final Assistant Response:", final_response)